In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [9]:

import torch.nn.functional as F

# Simple CNN model for colorization
class ColorizationNet(nn.Module):
    def __init__(self):
        super(ColorizationNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 2, 3, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.tanh(self.conv4(x))
        return x


In [ ]:
# Dataset preparation and training loop
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((32, 32))
])

# Using CIFAR-10 as example dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training parameters
model = ColorizationNet().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10

# Training loop
for epoch in range(epochs):
    for i, (inputs, _) in enumerate(train_loader):
        # Convert RGB to LAB and get grayscale (L) and color channels (ab)
        inputs = inputs.to(device)
        gray = inputs[:, 0:1, :, :]  # Using only one channel since grayscale
        
        # Forward pass
        outputs = model(gray)
        
        loss = criterion(outputs, torch.zeros_like(outputs))  # Placeholder
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Save model weights
torch.save(model.state_dict(), 'colorization_model.pth')

Epoch [1/10], Loss: 0.0000
Epoch [2/10], Loss: 0.0000
Epoch [3/10], Loss: 0.0000
Epoch [4/10], Loss: 0.0000
Epoch [5/10], Loss: 0.0000
Epoch [6/10], Loss: 0.0000
Epoch [7/10], Loss: 0.0000
Epoch [8/10], Loss: 0.0000


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import seaborn as sns

# Evaluation function (simplified example)
def evaluate_model(model, test_loader):
    model.eval()
    total = 0
    correct = 0
    
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            gray = inputs[:, 0:1, :, :]
            outputs = model(gray)
            
            _, predicted = torch.max(outputs.data, 1)
            total += 1  # Placeholder for proper evaluation
            correct += 1  # Placeholder
    
    accuracy = correct / total
    print(f'Accuracy: {accuracy:.2%}')
    
    # Confusion matrix would need proper classification setup
    # For colorization, other metrics like PSNR might be more appropriate
    return accuracy

# Prepare test dataset
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Evaluate
accuracy = evaluate_model(model, test_loader)

In [ ]:
import cv2
import numpy as np

def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (32, 32))  # assuming CIFAR-10 size
    gray_input = gray / 255.0
    gray_input = torch.tensor(gray_input).unsqueeze(0).unsqueeze(0).float().to(device)
    return gray_input, gray

def postprocess_output(output, gray_original):
    ab = output.squeeze().detach().cpu().numpy().transpose(1, 2, 0)
    ab = cv2.resize(ab, (gray_original.shape[1], gray_original.shape[0]))
    lab = np.zeros((gray_original.shape[0], gray_original.shape[1], 3))
    lab[:, :, 0] = gray_original
    lab[:, :, 1:] = ab * 128
    bgr = cv2.cvtColor(lab.astype(np.uint8), cv2.COLOR_LAB2BGR)
    return bgr


In [ ]:
import tkinter as tk
from threading import Thread

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load models
models = {
    "Model A": ColorizationNet().to(device),
    "Model B": ColorizationNet().to(device)  # For now same, later load different weights
}
try:
    models["Model A"].load_state_dict(torch.load('model_a_weights.pth'))
    models["Model B"].load_state_dict(torch.load('model_b_weights.pth'))
    print("Loaded model weights successfully")
except:
    print("Could not load weights, using random initialization")
current_model = list(models.values())[0]
cap = cv2.VideoCapture(0)  # Webcam

def update_frame():
    global current_model
    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        gray_input, gray_original = preprocess_frame(frame)
        with torch.no_grad():
            output = current_model(gray_input)
        colorized = postprocess_output(output, gray_original)

        cv2.imshow("Colorized Video", colorized)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def switch_model(name):
    global current_model
    current_model = models[name]

# GUI
root = tk.Tk()
root.title("Colorization Model Switcher")
tk.Label(root, text="Choose Colorization Model:").pack()

for name in models:
    btn = tk.Button(root, text=name, command=lambda n=name: switch_model(n))
    btn.pack()

# Run video capture in a thread
thread = Thread(target=update_frame, daemon=True)
thread.start()

root.mainloop()
